# Tarefa 02

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os monitores
- Divirta-se :)

#### 1. Carregue os dados da tarefa anterior

In [17]:
import pandas as pd
from plotly import graph_objects as go
from plotly.subplots import make_subplots

In [18]:
df = pd.read_csv('valores_bolsa.csv')
df.head()

,Price,Adj Close,Adj Close.1,Adj Close.2,Close,Close.1,Close.2,High,High.1,High.2,Low,Low.1,Low.2,Open,Open.1,Open.2,Volume,Volume.1,Volume.2
0,Ticker,DIS,JNJ,^DJI,DIS,JNJ,^DJI,DIS,JNJ,^DJI,DIS,JNJ,^DJI,DIS,JNJ,^DJI,DIS,JNJ,^DJI
1,Date,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-01-02,107.30461883544922,110.7385025024414,23346.240234375,108.97000122070312,127.75,23346.240234375,109.13999938964844,128.3800048828125,23413.470703125,107.7300033569336,126.27999877929688,22928.58984375,108.0999984741211,128.1300048828125,23058.609375,9723500,7631700,321570000
3,2019-01-03,104.70496368408203,108.97880554199219,22686.220703125,106.33000183105469,125.72000122070312,22686.220703125,108.6500015258789,128.27000427246094,23176.390625,105.94000244140625,125.36000061035156,22638.41015625,108.4800033569336,128.13999938964844,23176.390625,10594700,8654500,424240000
4,2019-01-04,107.9348373413086,110.8078384399414,23433.16015625,109.61000061035156,127.83000183105469,23433.16015625,110.75,128.64999389648438,23518.640625,107.25,126.7300033569336,22894.919921875,107.94000244140625,127.12000274658203,22894.919921875,10122800,8831700,396020000


**OBS:** O Dataframe foi salvo de maneira diferente daquela que ele foi importado, por isso, serão necessárias algumas etapas de tratamento dos dados.


#### 2. Média móvel

1. Faça uma média móvel  em 90 dias dos valores de fechamento ajustados dos ativos que você escolheu
2. Utilizando a fução ```rolling()```, calcule o desvio pardão desses ativos ao longo do tempo, utilizando a mesma janela móvel do item acima
2. Monte um gráfico de linhas para as médias móveis dos ativos da sua base
4. Monte um gráfico de linhas para os desvios padrão em janela móvel
5. Compare os dois gráficos que você fez com o da tarefa anterior. Veja se eles apontam conclusões semelhantes quanto aos períodos de maior volatilidade (variação) dos ativos.

In [19]:
df_roll_90 =  df.loc[df.index[2]:, ('Adj Close', 'Adj Close.1',	'Adj Close.2')].rolling(90)# Selecionando o período de 90 dias
# Agora iremos criar os nomes das colunas e extrair os valores delas do dataframe original.
media_movel = df_roll_90.mean().dropna()
media_movel.columns = ['DIS', 'JNJ', 'DJI']
desvio_padrao = df_roll_90.std().dropna()
desvio_padrao.columns = ['DIS', 'JNJ', 'DJI']

In [20]:
columns = pd.MultiIndex.from_product([['media_movel', 'desvio_padrao'], media_movel.columns])# Criando o índice hierárquico das colunas
data_graph = pd.DataFrame(index=df.Price.loc[91:],
                          columns=columns)

# Preenchendo o DataFrame com os valores
data_graph['media_movel'] = media_movel.values
data_graph['desvio_padrao'] = desvio_padrao.values
data_graph

media_movel                           desvio_padrao            \
                   DIS         JNJ           DJI           DIS       JNJ   
Price                                                                      
2019-05-10  115.277546  117.869679  25531.237609      9.393306  3.983731   
2019-05-13  115.522303  117.969461  25553.223720      9.469901  3.914980   
2019-05-14  115.816295  118.085111  25584.844054      9.540753  3.798409   
2019-05-15  116.090594  118.181311  25609.453602      9.665672  3.720878   
2019-05-16  116.363470  118.298013  25635.357270      9.805201  3.628026   
...                ...         ...           ...           ...       ...   
2024-02-23   92.861411  153.999247  36418.951345      7.846277  4.823832   
2024-02-26   93.108615  154.062460  36475.447917      7.959228  4.875430   
2024-02-27   93.369727  154.143537  36530.723047      8.104887  4.929424   
2024-02-28   93.663006  154.267704  36589.433507      8.253051  4.973972   
2024-02-29   93.980788  154.394469  36651.458160      8.386611  5.008504   

                         
                    DJI  
Price                    
2019-05-10   891.540154  
2019-05-13   860.923999  
2019-05-14   804.877531  
2019-05-15   771.518411  
2019-05-16   739.432183  
...                 ...  
2024-02-23  1928.752224  
2024-02-26  1931.111126  
2024-02-27  1930.588417  
2024-02-28  1922.791423  
2024-02-29  1909.200701  

[1210 rows x 6 columns]

In [21]:
# Transformaremos os valores do dataframe para o formato de dicionário, para serem plotados mais facilmente
data_x = data_graph.media_movel.reset_index(drop = True).values
transposed_data = list(map(list, zip(*data_x)))
dict_graph = {'DIS': transposed_data[0], 'JNJ': transposed_data[1], 'DJI': transposed_data[2]}

In [22]:
# Por fim, o gráfico será plotado, onde cada cor representa uma ação diferente
fig = go.Figure()
for chave, valores in dict_graph.items():
    fig.add_trace(go.Scatter(x=data_graph.index,
                             y=valores,
                             mode='lines',
                             name=chave))

fig.update_layout(title='Média Móvel',
                  xaxis_title='Datas',
                  yaxis_title='Valores')
fig.show()

In [23]:
# Faremos agora a mesma coisa para o desvio padrão
data_x_std = data_graph.desvio_padrao.reset_index(drop = True).values
transposed_data = list(map(list, zip(*data_x_std)))
dict_graph_std = {'DIS': transposed_data[0],
                  'JNJ': transposed_data[1],
                  'DJI': transposed_data[2]}

fig_std = go.Figure()
for chave, valores in dict_graph_std.items():
    fig_std.add_trace(go.Scatter(x=data_graph.index,
                                 y=valores,
                                 mode='lines',
                                 name=chave))
    
fig_std.update_layout(title='Desvio Padrão',
                  xaxis_title='Datas',
                  yaxis_title='Valores')

fig_std.show()

Em todos os gráficos, ambos os mostrados aqui juntamente com o mostrado no exercício passado, vemos que os valores da Dow Jones são tão grande que as outras duas ações parecem 'achatadas' na base dos gráficos. Além disso, ainda sobre a Dow Jones, vemos que em um período de grande apmlitude (entre janeiro e julho de 2020) temos também um grande desvio padrão positivo neste mesmo período, ou seja, ambos os gráficos apontam para conclusões semelhantes neste mesmo período. As outras ações também apresentam comportamento semelhante.

#### 3. Gráfico de *candlestick*
1. Selecione o período dos últimos 60 dias (corridos) para um dos ativos da sua base
2. Monte um gráfico de *candlestick* para esses dados.

In [24]:
# Primeiramente iremos montar um dataframe novo, apenas para os valores da Dow Jones, no período mencionado
df_dow_jones = (df[['Adj Close.2', 'Close.2', 'High.2', 'Low.2', 'Open.2']]
                .rename(columns={'Adj Close.2': 'Adj_close',
                                 'Close.2': 'Close',
                                 'High.2': 'High',
                                 'Low.2': 'Low',
                                 'Open.2': 'Open'})
                .iloc[2:].set_index(df.Price.loc[2:])
                .tail(60))


In [25]:
# Agora iremos plotar o gráfico candlestick, primeiramente criaremos um novo DataFrame para o gráfico
graph = {
    'x': df_dow_jones.index,
    'open': df_dow_jones.Open,
    'close': df_dow_jones.Close,
    'high': df_dow_jones.High,
    'low': df_dow_jones.Low,
}

# Criar figura de subplot
fig = make_subplots(rows=1, cols=1)

# Adicionar candlestick trace
fig.add_trace(go.Candlestick(x=graph['x'],
                             open=graph['open'],
                             high=graph['high'],
                             low=graph['low'],
                             close=graph['close'],
                             name='^DJI'),
              row=1, col=1)

# Atualizar layout
fig.update_layout(title="Índice Dow Jones",
                  xaxis_title="Data",
                  yaxis_title="Valor",
                  paper_bgcolor='rgba(255,255,255, 0.3)',
                  plot_bgcolor='rgba(255,255,255, 0.3)',
                  showlegend=True,
                  margin=dict(l=50, r=50, t=50, b=50))  # Aumentando a margem

# Atualizar estilo de grade
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey', tickformat=".2f")  # Especificando o formato dos rótulos

# Mostrar gráfico
fig.show()